In [ ]:
!pip install openpyxl

### Version Check

In [ ]:
# Check Pytorch installation
import torch, torchvision
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose
print('mmpose version:', mmpose.__version__)

# Check MMPose installation
import mmcv
print('mmcv version:', mmcv.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

### Imports

In [ ]:
# Copyright (c) OpenMMLab. All rights reserved.
import logging
import mimetypes
import os
import time
from argparse import ArgumentParser
from functools import partial
import pandas as pd
import csv


import cv2
import json_tricks as json
import mmcv
import mmengine
import numpy as np
from mmengine.logging import print_log

from mmpose.apis import (_track_by_iou, _track_by_oks,
                         convert_keypoint_definition, extract_pose_sequence,
                         inference_pose_lifter_model, inference_topdown,
                         init_model)
from mmpose.models.pose_estimators import PoseLifter
from mmpose.models.pose_estimators.topdown import TopdownPoseEstimator
from mmpose.registry import VISUALIZERS
from mmpose.visualization import FastVisualizer
from mmpose.structures import (PoseDataSample, merge_data_samples,
                               split_instances)
from mmpose.utils import adapt_mmdet_pipeline
from mmengine import ProgressBar

In [ ]:
# Configure the logging settings
logging.basicConfig(
    filename='data/pose/execution_times.log',  # Log file name
    filemode='a',                   # Append to the file ('w' for overwrite)
    level=logging.INFO,             # Log level
    format='%(asctime)s - %(levelname)s - %(message)s'  # Log format
)

### Initialize pose, detection and lifter models

In [ ]:
try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

# Configuration paths from 17 keypoints
det_config = 'mmpose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py'
det_checkpoint = 'https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth'
pose_estimator_config = 'mmpose/configs/body_2d_keypoint/rtmpose/body8/rtmpose-m_8xb256-420e_body8-256x192.py'
pose_estimator_checkpoint = 'https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth'
pose_lifter_config = 'mmpose/configs/body_3d_keypoint/video_pose_lift/h36m/video-pose-lift_tcn-243frm-supv-cpn-ft_8xb128-200e_h36m.py'
pose_lifter_checkpoint = 'https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth'

# Configuration paths from 133 keypoints
det_wholebody_config = 'mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_wholebody_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'   
pose_estimator_wholebody_config = 'mmpose/configs/wholebody_2d_keypoint/topdown_heatmap/coco-wholebody/td-hm_hrnet-w48_dark-8xb32-210e_coco-wholebody-384x288.py'
pose_estimator_wholebody_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth'
 
     
# Define parameters
show_visualization = True
save_predictions = False

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Initialize models
detector = init_detector(det_config, det_checkpoint, device=device)
detector.cfg = adapt_mmdet_pipeline(detector.cfg)

pose_estimator = init_model( pose_estimator_config,
                             pose_estimator_checkpoint,
                             device=device )

pose_estimator_wholebody = init_model( pose_estimator_wholebody_config,
                             pose_estimator_wholebody_checkpoint,
                             device=device )

assert isinstance(pose_estimator, TopdownPoseEstimator), 'Only "TopDown" model is supported for 2D pose detection'
assert isinstance(pose_estimator_wholebody, TopdownPoseEstimator), 'Only "TopDown" model is supported for 2D pose detection'

# Define visualizer settings
det_kpt_color = pose_estimator.dataset_meta.get('keypoint_colors', None)
det_dataset_skeleton = pose_estimator.dataset_meta.get('skeleton_links', None)
det_dataset_link_color = pose_estimator.dataset_meta.get('skeleton_link_colors', None)

pose_lifter = init_model(pose_lifter_config,
                         pose_lifter_checkpoint,
                         device=device )

assert isinstance(pose_lifter, PoseLifter), 'Only "PoseLifter" model is supported for 2D-to-3D lifting'

'''# Set up the visualizer. Works for the 3D
pose_lifter.cfg.visualizer.radius = 3
pose_lifter.cfg.visualizer.line_width = 1
pose_lifter.cfg.visualizer.det_kpt_color = det_kpt_color
pose_lifter.cfg.visualizer.det_dataset_skeleton = det_dataset_skeleton
pose_lifter.cfg.visualizer.det_dataset_link_color = det_dataset_link_color
visualizer = VISUALIZERS.build(pose_lifter.cfg.visualizer)
visualizer.set_dataset_meta(pose_lifter.dataset_meta) '''
# Init visualizer
pose_estimator_wholebody.cfg.visualizer.radius = 3
pose_estimator_wholebody.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator_wholebody.cfg.visualizer)
# The dataset_meta is loaded from the checkpoint and then passed to the model in init_pose_estimator
visualizer.set_dataset_meta(pose_estimator_wholebody.dataset_meta)

In [ ]:
#det_model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)

In [ ]:
'''det_model.cuda()
pose_model.cuda()
lifter_model.cuda()
None'''

#### Support functions

In [ ]:
def read_bboxes():
    ''' retrieve the bboxes that were added manually to the video'''
    bboxes = pd.read_csv('data/bboxes.csv')
    return bboxes

def get_video_reader(vidfile):
    """Initialize video reader and check fps."""
    vid_reader = mmcv.VideoReader(vidfile)
    if vid_reader.fps < 29 or vid_reader.fps > 31:
        print('fps = ' + str(vid_reader.fps))
    return vid_reader

def get_init_bbox(vidfile_name, bboxes):
    """Get the initial bounding box from the bounding boxes dataframe."""
    init_bbox = bboxes[bboxes['fname'] == vidfile_name].loc[:, 'xmin':'ymax']
    
    if init_bbox['xmin'].item() > init_bbox['xmax'].item():
        init_bbox['xmin'], init_bbox['xmax'] = init_bbox['xmax'], init_bbox['xmin']
    if init_bbox['ymin'].item() > init_bbox['ymax'].item():
        init_bbox['ymin'], init_bbox['ymax'] = init_bbox['ymax'], init_bbox['ymin']
    
    return init_bbox

def initialize_video_writer(output_path, width, height):
    """Initialize video writer for saving the output with pose visualization."""
    #print('Starting initialize_video_writer ...')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_writer = cv2.VideoWriter(output_path, fourcc, 30, (width, height))
    return vid_writer


#### Save pose to csv

In [ ]:
import numpy as np

def create_combined_data(result_pose, result_3d, bbox):
    """
    Combines data from result_pose, result_3d, and bbox into a single array.
    
    Args:
        result_pose (numpy.ndarray): Pose data.
        result_3d (numpy.ndarray): 3D data with dimensions (n, k, 3).
        bbox (array-like): Bounding box in [x_min, y_min, x_max, y_max] format.
    
    Returns:
        numpy.ndarray: Combined data array.
    """
    # Step 1: Flatten result_pose
    try:
        #print(f"result_ pose: {result_pose}")
        flattened_pose = result_pose.flatten()
        #print(f"Flattened pose: {flattened_pose}")
    except Exception as e:
        raise ValueError(f"Error flattening result_pose: {e}")

    # Step 2: Extract Z-coordinates from the first element of result_3d
    try:
        print(f"result_3d: {result_3d}")
        z_coords = result_3d[:, 2].tolist()
        print(f"Extracted Z-coordinates: {z_coords}")
    except Exception as e:
        print(f"Error extracting Z-coordinates from result_3d: {e}")

    # Step 3: Ensure bbox is a numpy array
    try:
        bbox_array = np.array(bbox)
        print(f"BBox array: {bbox_array}")
    except Exception as e:
        print(f"Error converting bbox to numpy array: {e}")

    # Step 4: Append all components
    try:
        combined_data = np.append(
            np.append(flattened_pose, z_coords),
            bbox_array
        )
        print(f"Combined data: {combined_data}")
    except Exception as e:
        print(f"Error combining data: {e}")
    
    return combined_data

# Example usage
result_pose = np.random.rand(17, 3)  # Example 17 keypoints with 3 coordinates each
result_3d = np.random.rand(1, 17, 3)  # Example 3D data (1 person, 17 keypoints, 3 coordinates)
bbox = [140, 44, 450, 450]  # Example bbox in [x_min, y_min, x_max, y_max] format

combined_data = create_combined_data(result_pose, result_3d, bbox)
print("Final combined data:", combined_data)


In [ ]:
def save_pose_results_to_csv(csv_writer, all_keypoints_wholebody, all_keypoints_3d, all_bboxes):
    """Save all pose and 3D results to the CSV file."""

    header = []
    for kpt in range(133):
        header.append(f'x{kpt}')
        header.append(f'y{kpt}')
        header.append(f'conf{kpt}')
    for kpt_3d in range(17):
        header.append(f'z{kpt_3d}') # this used to be kpt, which made me add a renaming step in the classifier
    header.append('xmin')
    header.append('ymin')
    header.append('xmax')
    header.append('ymax')
    header.append('bbox_conf')
    # write header
    csv_writer.writerow(header)
     
    # write data
    for result_pose, result_3d, bbox in zip(all_keypoints_wholebody, all_keypoints_3d, all_bboxes):
        try:
            # Create the row by combining data and adding a new value (e.g., new_value)
            combined_data = np.append(
                            np.append(result_pose.flatten(), result_3d[:, 2].tolist()), 
                            bbox)
            # Add conf score to the end
            combined_data = np.append(combined_data, 1)

            csv_writer.writerow(combined_data)
            
            #csv_writer.writerow(np.append(np.append(result_pose.flatten(), result_3d[0, :, 2].tolist()), bbox))
        except IndexError:
            #print(result_3d)
            raise Exception('something about dimensions')


#### Visualizer

In [ ]:
def visualize_and_save_frame(frame, data_sample_chosen, vid_writer,extra_bbox=None):
    #Visualize the pose results on a frame and save the frame to the video.

    data_samples = merge_data_samples([data_sample_chosen])

    visualizer.add_datasample(
        'result',
        frame,
        data_sample=data_samples,
        draw_gt=False,
        draw_heatmap=False,
        draw_bbox=True,
        show=False,
        wait_time=0,
        out_file=None,
        kpt_thr=0.3)

    processed_frame = visualizer.get_image()
    
    #vis_result_bgr = cv2.cvtColor(processed_frame, cv2.COLOR_RGB2BGR)

    # Add an extra bounding box if provided
    if extra_bbox is not None:
        # Draw the extra bounding box (distinct color, e.g., blue)
        # Use .iloc[0] to avoid FutureWarning
        xmin = int(extra_bbox['xmin'].iloc[0])
        ymin = int(extra_bbox['ymin'].iloc[0])
        xmax = int(extra_bbox['xmax'].iloc[0])
        ymax = int(extra_bbox['ymax'].iloc[0])
        cv2.rectangle(processed_frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)  # Blue box with thickness 2
    
    
    #cv2.imwrite('data/pose/videos/output_pose_image.png', vis_result_bgr)
    vid_writer.write(processed_frame)

#### Find 2d 133 keypoints

In [ ]:
# maybe remove this function move the 2d call into pose_estimation()
def get_2d_keypoints_from_frame(detector, frame, bbox_init, pose_estimator, visualizer):
    """ Find whole body keypoints of one image. 133 keypoints
    """
    # estimate pose results for current image
    #pose_est_results = inference_topdown(pose_estimator, frame, bboxes)
    pose_est_results = inference_topdown(pose_estimator, frame, np.array(bbox_init)[None],'xyxy')[0]
    
    return pose_est_results


#### Find 3d pose lifting

In [ ]:
def get_3d_keypoints_from_frame(detector, frame, frame_idx, pose_estimator,
                      pose_est_results_last, pose_est_results_list, next_id,
                      pose_lifter, visualize_frame, visualizer):
    """Visualize detected and predicted keypoints of one image.
    Pipeline -> Detect -> 2d pose estimate -> 3d pose lift
    works on 17 keypoints
    """
    
    pose_lift_dataset = pose_lifter.cfg.test_dataloader.dataset
    pose_lift_dataset_name = pose_lifter.dataset_meta['dataset_name']

    # First stage: conduct 2D pose detection in a Topdown manner
    # use detector to obtain person bounding boxes
    det_result = inference_detector(detector, frame)
    pred_instance = det_result.pred_instances.cpu().numpy()

    # filter out the person instances with category and bbox threshold
    # e.g. 0 for person in COCO
    bboxes = pred_instance.bboxes
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0,  # Assuming person category is 0
                                   pred_instance.scores > 0.3)]  # Example threshold

    # estimate pose results for current image
    pose_est_results = inference_topdown(pose_estimator, frame, bboxes)
    
    # Assuming you want to use IOU tracking
    _track = partial(_track_by_iou)  # Use IOU for tracking

    pose_det_dataset_name = pose_estimator.dataset_meta['dataset_name']
    #print(f"pose_det_dataset_name = {pose_det_dataset_name}")
    pose_est_results_converted = []

    # convert 2d pose estimation results into the format for pose-lifting
    # such as changing the keypoint order, flipping the keypoint, etc.
    for i, data_sample in enumerate(pose_est_results):
        pred_instances = data_sample.pred_instances.cpu().numpy()
        keypoints = pred_instances.keypoints
       
        # Trim keypoints to only the first 17
        #keypoints = keypoints[:, :17, :] #jerlin
        
        # calculate area and bbox
        if 'bboxes' in pred_instances:
            areas = np.array([(bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                              for bbox in pred_instances.bboxes])
            pose_est_results[i].pred_instances.set_field(areas, 'areas')
        else:
            areas, bboxes = [], []
            for keypoint in keypoints:
                xmin = np.min(keypoint[:, 0][keypoint[:, 0] > 0], initial=1e10)
                xmax = np.max(keypoint[:, 0])
                ymin = np.min(keypoint[:, 1][keypoint[:, 1] > 0], initial=1e10)
                ymax = np.max(keypoint[:, 1])
                areas.append((xmax - xmin) * (ymax - ymin))
                bboxes.append([xmin, ymin, xmax, ymax])
            pose_est_results[i].pred_instances.areas = np.array(areas)
            pose_est_results[i].pred_instances.bboxes = np.array(bboxes)

        # track id
        track_id, pose_est_results_last, _ = _track(data_sample,
                                                    pose_est_results_last,
                                                    0.3)  # Example tracking threshold
        if track_id == -1:
            if np.count_nonzero(keypoints[:, :, 1]) >= 3:
                track_id = next_id
                next_id += 1
            else:
                # If the number of keypoints detected is small,
                # delete that person instance.
                keypoints[:, :, 1] = -10
                pose_est_results[i].pred_instances.set_field(
                    keypoints, 'keypoints')
                pose_est_results[i].pred_instances.set_field(
                    pred_instances.bboxes * 0, 'bboxes')
                pose_est_results[i].set_field(pred_instances, 'pred_instances')
                track_id = -1
        pose_est_results[i].set_field(track_id, 'track_id')

        # convert keypoints for pose-lifting
        pose_est_result_converted = PoseDataSample()
        pose_est_result_converted.set_field(
            pose_est_results[i].pred_instances.clone(), 'pred_instances')
        pose_est_result_converted.set_field(
            pose_est_results[i].gt_instances.clone(), 'gt_instances')
        keypoints = convert_keypoint_definition(keypoints,
                                                pose_det_dataset_name,
                                                pose_lift_dataset_name)
        
        keypoints = [keypoints[0][:17]] #jerlin
        #print(f"keypoints trimmed = {[keypoints[0][:17]]}")  
        pose_est_result_converted.pred_instances.set_field(
            keypoints, 'keypoints')
        pose_est_result_converted.set_field(pose_est_results[i].track_id,
                                            'track_id')
        pose_est_results_converted.append(pose_est_result_converted)

    pose_est_results_list.append(pose_est_results_converted.copy())

    # Second stage: Pose lifting
    # extract and pad input pose2d sequence
    pose_seq_2d = extract_pose_sequence(
        pose_est_results_list,
        frame_idx=frame_idx,
        causal=pose_lift_dataset.get('causal', False),
        seq_len=pose_lift_dataset.get('seq_len', 1),
        step=pose_lift_dataset.get('seq_step', 1))

    # conduct 2D-to-3D pose lifting
    pose_lift_results = inference_pose_lifter_model(
        pose_lifter,
        pose_seq_2d,
        image_size=visualize_frame.shape[:2],
        norm_pose_2d=False)  # Example setting for norm_pose_2d

    # post-processing
    for idx, pose_lift_result in enumerate(pose_lift_results):
        pose_lift_result.track_id = pose_est_results[idx].get('track_id', 1e4)

        pred_instances = pose_lift_result.pred_instances
        keypoints = pred_instances.keypoints
        keypoint_scores = pred_instances.keypoint_scores
        if keypoint_scores.ndim == 3:
            keypoint_scores = np.squeeze(keypoint_scores, axis=1)
            pose_lift_results[
                idx].pred_instances.keypoint_scores = keypoint_scores
        if keypoints.ndim == 4:
            keypoints = np.squeeze(keypoints, axis=1)

        keypoints = keypoints[..., [0, 2, 1]]
        keypoints[..., 0] = -keypoints[..., 0]
        keypoints[..., 2] = -keypoints[..., 2]

        # rebase height (z-axis)
        keypoints[..., 2] -= np.min(
            keypoints[..., 2], axis=-1, keepdims=True)

        pose_lift_results[idx].pred_instances.keypoints = keypoints

    pose_lift_results = sorted(
        pose_lift_results, key=lambda x: x.get('track_id', 1e4))

    pred_3d_data_samples = merge_data_samples(pose_lift_results)
    det_data_sample = merge_data_samples(pose_est_results)
    pred_3d_instances = pred_3d_data_samples.get('pred_instances', None)

    # Assuming you want to visualize 1 3D pose
    num_instances = 1 

    # Visualization
    '''if visualizer is not None:
        visualizer.add_datasample(
            'result',
            visualize_frame,
            data_sample=pred_3d_data_samples,
            det_data_sample=det_data_sample,
            draw_gt=False,
            dataset_2d= pose_det_dataset_name,
            dataset_3d=pose_lift_dataset_name,
            show=False,
            draw_bbox=True,
            kpt_thr=0.3,  # Example threshold
            num_instances=num_instances,
            wait_time=0)'''

    return pose_est_results, pose_est_results_list, pred_3d_instances, next_id


#### calculate bbox overlap

In [ ]:
def calc_overlap(bbox_test, bbox_init):
    '''returns overlap between test bbox and init bbox as a fraction of init bbox.
    requires min <= max'''

    bbox_test = bbox_test[0]
    
    # Assume `bbox_init` is a pandas DataFrame, extract values to array or list
    bbox_init_values = bbox_init.to_numpy()
    bbox_init = bbox_init_values[0]
    
    # Calculate overlap
    overlap_width = min(bbox_init[1], bbox_test[2]) - max(bbox_init[0], bbox_test[0])
    overlap_height = min(bbox_init[3], bbox_test[3]) - max(bbox_init[2], bbox_test[1])
    if overlap_width < 0 or overlap_height < 0:
        return 0
    return (overlap_width * overlap_height) / ((bbox_init[1] - bbox_init[0]) * (bbox_init[3] - bbox_init[2]))


### Pose Estimation

In [ ]:
print(f"Device = {device}")
def pose_estimate(vidfile, start, end):
    """Main function to estimate poses in a video."""
    vid_reader = get_video_reader(vidfile)
    print(f"Processing {vidfile}")
    clip_name = vidfile[-len('7942GT00.mp4'):-len('.mp4')]
    csv_fname = "data/pose/" + clip_name + '_dbx.csv'
    if os.path.exists(csv_fname):
        print(csv_fname + ' already exists. Skipping for now')
        return

    vidfile_name = vidfile.split('/')[-1]
    bboxes = read_bboxes()
    init_bbox = get_init_bbox(vidfile_name, bboxes) 
    # Assume `bbox_init` is a pandas DataFrame, extract values to array or list
    bbox_init_values = init_bbox.to_numpy()
    bbox_init = bbox_init_values[0]
    #convert to xyxy
    bbox_init = [bbox_init[0], bbox_init[2], bbox_init[1], bbox_init[3]]
    #print(f"bbox_init={bbox_init}")
    
    vid_writer = initialize_video_writer(f'data/pose/videos/{clip_name}.mp4', vid_reader.width, vid_reader.height)

    #from tqdm import tqdm
    total_frames = min(int(end * vid_reader.fps), len(vid_reader)) #tqdm
    progress_bar = ProgressBar(task_num = total_frames)

    with open(csv_fname, 'w') as csvfile:
        csv_writer = csv.writer(csvfile)

        frame_num = 0
        all_keypoints_wholebody = []
        all_keypoints_3d = []
        all_bboxes = []

        while frame_num < total_frames:#tqdm
            negative_frame = vid_reader[frame_num] #tqdm
            frame_num += 1 #tqdm
            #print(f"frame number = {frame_num}")
            # Convert frame to RGB for inference
            frame = mmcv.imread(negative_frame, channel_order='rgb')
            progress_bar.update()

            start_time = time.time()
             # Process image and perform 2D pose estimation on whole body
            pose_est_results = get_2d_keypoints_from_frame(detector=detector,
                                                                      frame=frame,
                                                                      bbox_init=bbox_init,
                                                                      pose_estimator=pose_estimator_wholebody,
                                                                      visualizer=visualizer)


            elapsed_time = time.time() - start_time
            logging.info(f"Code block get_2d_keypoints_from_frame took {elapsed_time:.2f} seconds")
            #mmcv.imwrite(mmcv.rgb2bgr(processed_frame), "data/pose/output_image.jpg")

             
            keypoint_scores_chosen = pose_est_results.pred_instances.keypoint_scores[0]
            keypoints_wholebody_chosen = pose_est_results.pred_instances.keypoints[0]
            bbox_chosen = pose_est_results.pred_instances.bboxes[0]
            data_sample_chosen = pose_est_results
            
            scores_expanded = np.expand_dims(keypoint_scores_chosen, axis=-1)
            keypoints_wholebody = np.concatenate([keypoints_wholebody_chosen, scores_expanded], axis=-1)
                
            start_time = time.time()
            # Process image and perform 3D pose lifting
            pose_est_results_list = []
            _, _, pred_3d_instances, _ = get_3d_keypoints_from_frame(detector=detector,
                                                                      frame=frame,
                                                                      frame_idx=0,
                                                                      pose_estimator=pose_estimator,
                                                                      pose_est_results_last=[],
                                                                      pose_est_results_list=pose_est_results_list,
                                                                      next_id=0,
                                                                      pose_lifter=pose_lifter,
                                                                      visualize_frame=frame,
                                                                      visualizer=visualizer)

            elapsed_time = time.time() - start_time
            logging.info(f"Code block get_3d_keypoints_from_frame took {elapsed_time:.2f} seconds")

            keypoints_3d = pred_3d_instances.keypoints[0]
            
            if len(keypoints_wholebody_chosen > 0):
                all_keypoints_wholebody.append(keypoints_wholebody)
                all_keypoints_3d.append(keypoints_3d)
                all_bboxes.append(bbox_chosen)
            else:
                all_keypoints_wholebody.append(np.zeros(133))
                all_keypoints_3d.append(np.zeros((17, 4)))
                all_bboxes.append(np.zeros(4))

            # Visualize and save each frame to the video
            start_time = time.time()
            visualize_and_save_frame(frame, data_sample_chosen, vid_writer, init_bbox)
            #plot_keypoints_opencv(frame, keypoints_wholebody_chosen, vid_writer, bbox_chosen, init_bbox)
            elapsed_time = time.time() - start_time
            logging.info(f"Code block visualize_and_save_frame took {elapsed_time:.2f} seconds")
            
        vid_writer.release()  # Release the video writer
        
        # Save pose results to CSV
        save_pose_results_to_csv(csv_writer, all_keypoints_wholebody, all_keypoints_3d, all_bboxes)


In [ ]:
video_data = pd.read_excel('data/EEG Video Timings.xlsx')
type_col_name = list(filter(lambda s: s.startswith('Type'), video_data.columns))[0]
generalized_sz = video_data[video_data[type_col_name] == 0]
fnames = 'data/videos/' + generalized_sz['Filename']+'.mp4'

### Run

In [ ]:
for fname, start, end in zip(fnames, generalized_sz['Video\nT0 to Start'], generalized_sz['Video\nT0 to End']):
    try:
        start_sec = int(start.split(':')[0]) * 60 + int(start.split(':')[1])
        end_sec = int(end.split(':')[0]) * 60 + int(end.split(':')[1])
        pose_estimate(fname, start_sec, end_sec)
    except ValueError as exc:
        print('No timings available for video file: ' + fname)
    except FileNotFoundError:
        print('Could not find video file: ' + fname)